# Homework 6
Josh Cohen

In [65]:
import numpy as np
import sympy as sym
import pandas as pd
%matplotlib notebook
from matplotlib import pyplot as plt
from scipy.optimize import minimize
from scipy.integrate import solve_ivp, solve_bvp, trapz

## Problem 1
Demonstrate numerically that the optimal solution obtained by the Riccati equation form Homework 5 problem 4 is indeed optimal. Evaluate the directional derivative of $J$ at $(x_{sol}(t), u_{sol}(t))$ in 10 different directions. Make a table of the 10 directions and the corresponding value of the directional derivative.

In [162]:
optSoln = pd.read_csv("optSolnP4.csv", header=0, index_col=0)

In [163]:
x_opt = optSoln.loc["x0":"x1"].to_numpy()
u_opt = np.expand_dims(optSoln.loc["u"].to_numpy(),0)
t_vec = optSoln.loc["t"].to_numpy()

In [164]:
t, A_sym, B_sym, C_sym, D_sym = sym.symbols('t A B C D')
v_gen = A_sym * sym.sin(B_sym*t + C_sym) + D_sym

In [165]:
def get_rand_v():
    randos = np.random.rand(4)
    return v_gen.subs({A_sym:randos[0], B_sym:randos[1], C_sym:randos[2], D_sym:randos[3]})

In [174]:
Q = sym.Matrix([[2,0],[0,0.01]])
R = sym.Matrix([0.1])
P1 = sym.Matrix([[1,0],[0,0.01]])

x0 = sym.Function('x_0')(t)
x1 = sym.Function('x_1')(t)
z0 = sym.Function('z_0')(t)
z1 = sym.Function('z_1')(t)

x = sym.Matrix([x0, x1])
z = sym.Matrix([z0, z1])

x_fin = sym.MatrixSymbol('x(T)', 2, 1).as_explicit()
z_fin = sym.MatrixSymbol('z(T)', 2, 1).as_explicit()
x_init = sym.Matrix([10,0]) #possibly want this to be zeros
z_init = np.zeros((2,1))

A = sym.Matrix([[0,1],[-1.6,-0.4]])
B = sym.Matrix([0,1]) 

u0 = sym.Function('u_0')(t)
v0 = sym.Function('v_0')(t)
u = sym.Matrix([u0])
v = sym.Matrix([v0])
b = u.T * R

xdot = A * x + B * u

p = sym.Matrix([sym.Function('p_0')(t), sym.Function('p_1')(t)])
p_fin = sym.MatrixSymbol('p_{1}', 2, 1).as_explicit()
pdot = -A.T * p - Q*x

In [214]:
v_pert = sym.Matrix([get_rand_v()])
pert_init = np.concatenate((z_init, v_pert.subs(t,0)))
zdot = A * z + B * v_pert 
pert = sym.Matrix([z,v])
zdot_lam = sym.lambdify([t, pert], sym.flatten(sym.Matrix([zdot, v_pert])))

In [215]:
ans = solve_ivp(zdot_lam, (0, 10), pert_init.flatten(), t_eval=t_vec)

In [216]:
zed = ans.y[:2,:]
v_lam = sym.lambdify(t,v_pert)
ved = np.expand_dims(v_lam(t_vec).flatten(),0)

In [217]:
J_dd_integrand = np.zeros(zed.shape[1])
for i in range(zed.shape[1]) :
    J_dd_integrand[i] = np.matmul(np.matmul(x_opt[:,i].T, Q), zed[:,i]) \
                                    + np.matmul(np.matmul(u_opt[:,i].T, R), ved[:,i])

In [218]:
J_dd_term = np.matmul(np.matmul(x_opt[:,-1].T, P1), zed[:,-1])

In [219]:
trapz(J_dd_integrand, t_vec) + J_dd_term

-0.00963582260126982

# Probelm 2
Compute the control $u(t)$ that minimizes the given cost function subject to the given constraint and desired trajectory